In [1]:
import os
from pydub import AudioSegment, silence
import wave
from google.cloud import speech
from google.cloud import storage
from concurrent.futures import ThreadPoolExecutor
import io

from pyannote.audio import Pipeline

pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'Speech2Text.json'

## Tạo thư mục cho từng tác vụ
Thu_muc_cho_audio_cat_theo_giong_noi = 'Splited_speaker'
Thu_muc_cho_audio_cat_theo_silence   = 'Splited_Silence'

os.makedirs(Thu_muc_cho_audio_cat_theo_giong_noi ,exist_ok= True)
os.makedirs(Thu_muc_cho_audio_cat_theo_silence   ,exist_ok= True)

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [2]:
%cd C:\Users\quang\Desktop\chominh\s2t_GCP_Final

c:\Users\quang\Desktop\chominh\s2t_GCP_Final


In [14]:
path = './Input/demo_1_29070307.wav'
audio_file = os.path.abspath(path)
audio_file

'c:\\Users\\quang\\Desktop\\chominh\\s2t_GCP_Final\\Input\\demo_1_29070307.wav'

In [15]:
audio_file

'c:\\Users\\quang\\Desktop\\chominh\\s2t_GCP_Final\\Input\\demo_1_29070307.wav'

In [16]:
Audio = AudioSegment.from_file(audio_file)
name = Thu_muc_cho_audio_cat_theo_giong_noi + '\\' + audio_file.split('\\')[-1].replace('.wav', '')
name

'Splited_speaker\\demo_1_29070307'

In [17]:
os.makedirs(name, exist_ok=True)

In [18]:
hashDict = dict()
diarization = pipeline(audio_file)

In [20]:
hashDict

{}

In [14]:
name

'Splited_speaker\\Recording2_Reduce_Noise'

In [21]:
i,start,end,lab = 0,0,0,0
for segment,track,label in diarization.itertracks(yield_label = True):
    if label != lab:
        Audio_speaker_i = Audio[start * 1000 : end * 1000]
        Audio_speaker_i.export(name + '\\' + str(i) + '.wav', format = 'wav')
        hashDict[name + '\\' + str(i) + '.wav'] = lab
        i += 1
        start = segment.start
        end = segment.end
        lab = label
    elif label == lab:
        end = segment.end
Audio_speaker_i = Audio[start*1000 : end*1000]
Audio_speaker_i.export(name + '\\' +str(i) + '.wav',format='wav')
hashDict[name + '\\' + str(i) + '.wav'] = lab
os.remove(name + '\\' + str(0) + '.wav')
hashDict.pop(name + '\\' +str(0) + '.wav')


0

In [22]:
hashDict

{'Splited_speaker\\demo_1_29070307\\1.wav': 'SPEAKER_00',
 'Splited_speaker\\demo_1_29070307\\2.wav': 'SPEAKER_03',
 'Splited_speaker\\demo_1_29070307\\3.wav': 'SPEAKER_00',
 'Splited_speaker\\demo_1_29070307\\4.wav': 'SPEAKER_02',
 'Splited_speaker\\demo_1_29070307\\5.wav': 'SPEAKER_00',
 'Splited_speaker\\demo_1_29070307\\6.wav': 'SPEAKER_01'}

In [17]:
def export_audio(audio, count,name):
    audios = audio.set_frame_rate(16000)
    audios.export(os.path.join(name + '\\file_{}.wav'.format(str(count))), format='wav')

folder = Thu_muc_cho_audio_cat_theo_silence
os.makedirs(folder, exist_ok=True)
def split_silence(audio, speaker):
    folder = Thu_muc_cho_audio_cat_theo_silence
    os.makedirs(folder, exist_ok=True)
    name = audio.split('\\')
    name = name[-2] + '\\' + name[-1].replace('.wav', '')
    name = os.path.join(folder, name)
    os.makedirs(name, exist_ok=True)

    myaudio = AudioSegment.from_file(audio, "wav")
    dbfs = myaudio.dBFS
    duration_in_sec = len(myaudio) / 1000

    mydict = dict()
    t_dict = dict()

    # Lấy các khoảng silence trong audio
    silences = silence.detect_silence(myaudio,
                                      min_silence_len= 300,
                                      silence_thresh=dbfs-10)
    silences = [((start/1000),(stop/1000)) for start,stop in silences]
    # print(silences)

    if len(silences) > 0:
        n_silence = []
        if silences[0][0] == 0.0:
            n_silence.append(silences[0])
            silences.pop(0)

        # Chỉnh lại, làm tròn sec
        for i in silences:
            if round(i[0]) < i[0]:
                temp= (i[0]+0.5, i[1])
            else:
                temp= (round(i[0]), i[1])
            n_silence.append(temp)
        

        count = 1
        for start,end in silences:
            temp= name+'\\file_'+str(count)+'.wav' # vị trí file: lưu file ở thư mục nào thì địa chỉ tới thư mục đó
            t_dict[temp]= end-start
            count +=1
        

        start = 0.0
        end = duration_in_sec
        s_audio = myaudio[start*1000:n_silence[0][0]*1000]
        export_audio(s_audio, 1,name)
        count = 2
        for i in range(len(n_silence)-1):
            s= n_silence[i][1]
            e= n_silence[i+1][0]
            n_audio = myaudio[s*1000:e*1000]
            export_audio(n_audio, count,name)
            count += 1
        if n_silence[len(n_silence)-1][1] != end:
            e_audio = myaudio[n_silence[len(n_silence)-1][1]*1000:end*1000]
            export_audio(e_audio, count,name)
            temp= name + '\\file_'+str(count)+'.wav'
            t_dict[temp] = 0
    else:
        temp= name+'\\file_'+str(0)+'.wav'
        export_audio(myaudio, 0,name)
        t_dict[temp] = 0
    mydict[speaker] = t_dict
    return mydict
    

In [18]:
def list_buckets():
    """Lists all buckets."""

    storage_client = storage.Client()
    buckets = storage_client.list_buckets()

    for bucket in buckets:
        print(bucket.name)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

In [19]:
# Một số Function hỗ trợ xử lý văn bản đầu ra
def is_digit(word):
    try:
        int(word)
        return True
    except ValueError:
        pass
    return False


def ConvertDate(text):
    month=' tháng '
    year=' năm '
    for index in range(0,len(text)):
        try:
            if (text.index(month,index)==index):
                dateNum = text[index -1]
                monthNum = text[index + len(month)]
                if is_digit(dateNum) and is_digit(monthNum):
                    text=text[:index] + text[index+len(month)-1:]
                    temp = list(text)
                    temp[index]='/'
                    text = "".join(temp)
        except Exception as e:
            if str(e) in 'substring not found':
                pass
            else:
                raise e
        try:
            if (text.index(year,index)==index):
                monthNum = text[index -1]
                yearNum = text[index + len(year)]
                if is_digit(monthNum) and is_digit(yearNum):
                    text=text[:index] + text[index+len(year)-1:]
                    temp = list(text)
                    temp[index]='/'
                    text = "".join(temp)
        except Exception as e:
            if str(e) in 'substring not found':
                pass
            else:
                raise e
    return text

In [20]:
# Các Function xử dụng GCP API thực hiện speech to text
def frame_rate_channel(audio_file_name):
    print(audio_file_name)
    with wave.open(audio_file_name, "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        channels = wave_file.getnchannels()
        return frame_rate,channels

## Config_GGC_model_before_Transcribe
def Config_GGC(sample_rate_hertz = 44100,
                audio_channel_count = 1):
        config_wav_enhanced = speech.RecognitionConfig(
            sample_rate_hertz = sample_rate_hertz,
            language_code = 'vi-VN',
            audio_channel_count=audio_channel_count,
        )
        
        return config_wav_enhanced



def Transcribe_Short_Audio(Audio_wav,config_wav_enhanced):
    client = speech.SpeechClient()
    with io.open(Audio_wav, "rb") as audio_file:
        content = audio_file.read()

    audio = speech.RecognitionAudio(content=content)
    # print(type(audio))
    
    response = client.recognize(config=config_wav_enhanced, audio=audio)
    text = []
    for i, result in enumerate(response.results):
        alternative = result.alternatives[0]
        alter=ConvertDate(alternative.transcript+ '\n')
        text.append(alter)
    return text

### This function is main on stranscribe
def Transcribe_Long_Audio(Audio_wav,config_wav_enhanced,
                        bucket_name = 'speech_to_text_stech',
                        Name = 'Audio_wav'):
    client = speech.SpeechClient()
    Audio_name = Audio_wav.split('\\')[-1].split('.')[0]
    
    upload_blob(bucket_name,Audio_wav,Name)
    
    media_uri = "gs://{}/{}".format(bucket_name,Name)
    long_audi_wav = speech.RecognitionAudio(uri=media_uri)
    
    
    operations = client.long_running_recognize(
        config = config_wav_enhanced,
        audio = long_audi_wav
    )
    
    response = operations.result(timeout=90)

    text = []
    
    for i, result in enumerate(response.results):
            alternative = result.alternatives[0]
            alter=ConvertDate(alternative.transcript)
            text.append(alter)
    return text

In [21]:
##Take audio to text+punc
def AudioToText(path,config):
    try:
        text=Transcribe_Short_Audio(path,config)
    except:
        text = Transcribe_Long_Audio(path,config)
    return text

In [22]:
hashDict

{'Splited_speaker\\Recording2_Reduce_Noise\\1.wav': 'SPEAKER_00',
 'Splited_speaker\\Recording2_Reduce_Noise\\2.wav': 'SPEAKER_02',
 'Splited_speaker\\Recording2_Reduce_Noise\\3.wav': 'SPEAKER_00',
 'Splited_speaker\\Recording2_Reduce_Noise\\4.wav': 'SPEAKER_01',
 'Splited_speaker\\Recording2_Reduce_Noise\\5.wav': 'SPEAKER_00',
 'Splited_speaker\\Recording2_Reduce_Noise\\6.wav': 'SPEAKER_02',
 'Splited_speaker\\Recording2_Reduce_Noise\\7.wav': 'SPEAKER_00'}

In [23]:
My_info = []
for audio, speaker in hashDict.items():
    my_dict = split_silence(audio,speaker)
    My_info.append(my_dict)

In [24]:
My_info

[{'SPEAKER_00': {'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_1.wav': 0.8319999999999999,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_2.wav': 1.3479999999999999,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_3.wav': 0.5670000000000002,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_4.wav': 0.39200000000000124,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_5.wav': 0.379999999999999,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_6.wav': 1.0580000000000016,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_7.wav': 0.5109999999999992,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_8.wav': 0.3440000000000012,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_9.wav': 0}},
 {'SPEAKER_02': {'Splited_Silence\\Recording2_Reduce_Noise\\2\\file_1.wav': 0.542,
   'Splited_Silence\\Recording2_Reduce_Noise\\2\\file_2.wav': 1.1050000000000004,
   'Splited_Silence\\Recording2_Reduce_Noise\\2\\file_3.wav': 1.0070000000000014,
   'Splited_Sile

In [25]:
##Take audio to text+punc
def AudioToText(path,config):
    try:
        text=Transcribe_Short_Audio(path,config)
    except:
        text = Transcribe_Long_Audio(path,config)
    return text

In [34]:
speaker = list(My_info[0])[0]
sample_file_config = list(My_info[0][speaker])[0]
rate,channel=frame_rate_channel(sample_file_config)
rate, channel

Splited_Silence1\test2\1\file_0.wav


(16000, 2)

In [35]:
config = Config_GGC(sample_rate_hertz = rate,
                 audio_channel_count = channel)

In [36]:
text = AudioToText(sample_file_config, config)
text

[]

In [37]:
x = speaker, text
x

('SPEAKER_00', [])

In [38]:
result_dict = {'Speaker':[], 'co dau':[],'khong dau':[],'silence':[]}

In [39]:
My_info

[{'SPEAKER_02': {'Splited_Silence\\test\\1\\file_1.wav': 0.4570000000000001,
   'Splited_Silence\\test\\1\\file_2.wav': 2.021,
   'Splited_Silence\\test\\1\\file_3.wav': 0.3949999999999996,
   'Splited_Silence\\test\\1\\file_4.wav': 0.38200000000000145,
   'Splited_Silence\\test\\1\\file_5.wav': 0.3280000000000012,
   'Splited_Silence\\test\\1\\file_6.wav': 0.43400000000000105,
   'Splited_Silence\\test\\1\\file_7.wav': 0.44899999999999807,
   'Splited_Silence\\test\\1\\file_8.wav': 0.4559999999999995,
   'Splited_Silence\\test\\1\\file_9.wav': 0.40700000000000003,
   'Splited_Silence\\test\\1\\file_10.wav': 0.36599999999999966,
   'Splited_Silence\\test\\1\\file_11.wav': 0.8339999999999996,
   'Splited_Silence\\test\\1\\file_12.wav': 0.7150000000000034,
   'Splited_Silence\\test\\1\\file_13.wav': 0.34899999999999665,
   'Splited_Silence\\test\\1\\file_14.wav': 0.36599999999999966}},
 {'SPEAKER_01': {'Splited_Silence\\test\\2\\file_1.wav': 0.6280000000000001,
   'Splited_Silence\\test\

In [54]:
index = list(My_info[0])[0]
list(My_info[0][index])[0]

'Splited_Silence\\test\\1\\file_1.wav'

In [59]:
My_info

[{'SPEAKER_02': {'Splited_Silence\\test\\1\\file_1.wav': 0.4570000000000001,
   'Splited_Silence\\test\\1\\file_2.wav': 2.021,
   'Splited_Silence\\test\\1\\file_3.wav': 0.3949999999999996,
   'Splited_Silence\\test\\1\\file_4.wav': 0.38200000000000145,
   'Splited_Silence\\test\\1\\file_5.wav': 0.3280000000000012,
   'Splited_Silence\\test\\1\\file_6.wav': 0.43400000000000105,
   'Splited_Silence\\test\\1\\file_7.wav': 0.44899999999999807,
   'Splited_Silence\\test\\1\\file_8.wav': 0.4559999999999995,
   'Splited_Silence\\test\\1\\file_9.wav': 0.40700000000000003,
   'Splited_Silence\\test\\1\\file_10.wav': 0.36599999999999966,
   'Splited_Silence\\test\\1\\file_11.wav': 0.8339999999999996,
   'Splited_Silence\\test\\1\\file_12.wav': 0.7150000000000034,
   'Splited_Silence\\test\\1\\file_13.wav': 0.34899999999999665,
   'Splited_Silence\\test\\1\\file_14.wav': 0.36599999999999966}},
 {'SPEAKER_01': {'Splited_Silence\\test\\2\\file_1.wav': 0.6280000000000001,
   'Splited_Silence\\test\

In [26]:
def GCP_s2t(alist):
    speaker = list(alist[0])[0]
    sample_file_config=list(alist[0][speaker])[0]
    rate,channel=frame_rate_channel(sample_file_config)
    config = Config_GGC(sample_rate_hertz = rate,
                 audio_channel_count = channel)
    def thread_function(index, file_addr):
        text = AudioToText(file_addr, config)
        x = index, text
        return x
    def func_sort(e):
        return e[0]
    result_dict = {'Speaker':[], 'co dau':[],'khong dau':[],'silence':[]}
    for adict in alist:
        for speaker, value in adict.items():
            file_addr = list(adict[speaker].keys())
            silent_time = list(adict[speaker].values())
            index = list(range(len(file_addr)))
            with ThreadPoolExecutor(8) as executor:
                futures = executor.map(thread_function,index,file_addr)
                index_text = [x for x in futures] 
            
            index_text.sort(key=func_sort)
            text = [text for _,text in index_text]

            result_dict['Speaker'].extend([speaker] * len(silent_time))
            result_dict['co dau'].extend(text)
            result_dict['khong dau'].extend(text)
            result_dict['silence'].extend(silent_time)
    return result_dict


In [27]:
Text_Dict=GCP_s2t(My_info)
Text_Dict

Splited_Silence\Recording2_Reduce_Noise\1\file_1.wav


{'Speaker': ['SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_00',
  'SPEAKER_02',
  'SPEAKER_02

In [28]:
Text_Dict['khong dau']

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [29]:
My_info

[{'SPEAKER_00': {'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_1.wav': 0.8319999999999999,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_2.wav': 1.3479999999999999,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_3.wav': 0.5670000000000002,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_4.wav': 0.39200000000000124,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_5.wav': 0.379999999999999,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_6.wav': 1.0580000000000016,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_7.wav': 0.5109999999999992,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_8.wav': 0.3440000000000012,
   'Splited_Silence\\Recording2_Reduce_Noise\\1\\file_9.wav': 0}},
 {'SPEAKER_02': {'Splited_Silence\\Recording2_Reduce_Noise\\2\\file_1.wav': 0.542,
   'Splited_Silence\\Recording2_Reduce_Noise\\2\\file_2.wav': 1.1050000000000004,
   'Splited_Silence\\Recording2_Reduce_Noise\\2\\file_3.wav': 1.0070000000000014,
   'Splited_Sile

In [30]:
speaker=list(My_info[0])[0]
sample_file_config=list(My_info[0][speaker])[0]
rate,channel=frame_rate_channel(sample_file_config)
config = Config_GGC(sample_rate_hertz = rate,
                 audio_channel_count = channel)
def thread_function(index,file_addr):
        text=AudioToText(file_addr,config)
        x = index,text
        return x
def func_sort(e):
        return e[0]
result_dict = {'Speaker':[], 'co dau':[],'khong dau':[],'silence':[]}
for adict in My_info:
    for speaker, value in adict.items():
        file_addr = list(adict[speaker].keys())
        silent_time = list(adict[speaker].values())
        index = list(range(len(file_addr)))
        with ThreadPoolExecutor(8) as executor:
            futures = executor.map(thread_function, index, file_addr)
            index_text = [x for x in futures]
        index_text.sort(key=func_sort)
        text = [text for _,text in index_text]
        result_dict['Speaker'].extend([speaker] * len(silent_time))
        result_dict['co dau'].extend(text)
        result_dict['khong dau'].extend(text)
        result_dict['silence'].extend(silent_time)
result_dict


Splited_Silence\Recording2_Reduce_Noise\1\file_1.wav


{'Speaker': ['SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_02',
  'SPEAKER_00',
  'SPEAKER_00',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_01',
  'SPEAKER_00',
  'SPEAKER_02',
  'SPEAKER_02

In [31]:
def Handle_Text_Dict(adict,S_Punc_ok=False):
    adict = list(adict.values())
    Spr,SL = 0,0
    ReDict = {'Speaker':[], 'co dau':[],'khong dau':[]}
    Silent_Punc = []
    for Speaker,Punc,NoPunc,Silent in zip(*adict):
        # Vì Punc,NoPunc là list
        new_Punc = str()
        for i in Punc:
            new_Punc += i
        new_NoPunc = str()
        for i in NoPunc:
            new_NoPunc += i
        
        if Speaker != Spr:
            # Làm gì đó
            Spr = Speaker
            SL = Silent
            ReDict['Speaker'].append(Speaker)
            
            ReDict['khong dau'].append(new_NoPunc)
            if S_Punc_ok == True:
                ReDict['co dau'].append(new_NoPunc + '. ')
            else:
                ReDict['co dau'].append(new_Punc)
        else: 
            ReDict['khong dau'][-1] += ' ' + new_NoPunc
            ReDict['co dau'][-1] += ' ' + new_Punc
    return ReDict

In [32]:
Result = Handle_Text_Dict(Text_Dict)
Result2 = Handle_Text_Dict(Text_Dict,S_Punc_ok=True)    

In [33]:
Result

{'Speaker': ['SPEAKER_00',
  'SPEAKER_02',
  'SPEAKER_00',
  'SPEAKER_01',
  'SPEAKER_00',
  'SPEAKER_02',
  'SPEAKER_00'],
 'co dau': ['        ',
  '                  ',
  ' ',
  '                            ',
  '',
  '             ',
  '                                                                          '],
 'khong dau': ['        ',
  '                  ',
  ' ',
  '                            ',
  '',
  '             ',
  '                                                                          ']}

In [1]:
from pyannote.core import Segment, Timeline, Annotation

In [2]:
from pyannote.metrics.diarization import DiarizationErrorRate
metric = DiarizationErrorRate()
import numpy as np
from pyannote.metrics.binary_classification import det_curve
from pyannote.metrics.binary_classification import precision_recall_curve

In [4]:
import matplotlib
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    matplotlib.use('Agg')
import matplotlib.pyplot as plt